# Setup

In [ ]:
# Mount data
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Files' location, set to your object-detection-zoo-master repository
%cd 'drive/My Drive/Colab Notebooks/object-detection-zoo-master/'

/content/drive/My Drive/Colab Notebooks/object-detection-zoo-master


In [ ]:
# Run this Cell when start a new kernel
# This cell can be commented out after running once
# Set up vertual environment
!pip install virtualenv
!virtualenv -p /usr/bin/python3 bigdata
!source bigdata/bin/activate

# Fix bugs and install dependencies
!pip uninstall -y tensorflow
!pip install -r requirements.txt


     |████████████████████████████████| 4.9MB 5.4MB/s 
     |████████████████████████████████| 337kB 57.7MB/s 
created virtual environment CPython3.6.9.final.0-64 in 19454ms
  creator CPython3Posix(dest=/content/drive/My Drive/Colab Notebooks/object-detection-zoo-master/bigdata, clear=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==20.2.3, pip==20.2.4, setuptools==50.3.0, setuptools==50.3.1, setuptools==50.3.2, wheel==0.35.1
  activators BashActivator,CShellActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator
Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 14.5MB 198kB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 377.0MB 23kB/s 
     |████████████████████████████████| 317kB 40.2MB/s 
     |████████████████████

In [ ]:
from general_utils.processing import letterbox_image_padded, decode_detection_raw
from general_utils.visualization import visualize_detection

import json
import cv2
import time
from PIL import Image
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()


In [ ]:
# If meet wierd tensorflow bugs
# Try to unintall and install tensorflow-gpu 

# !pip uninstall tensorflow-gpu==1.14.0
# !pip install tensorflow-gpu==1.14.0

# Convert video to images

In [ ]:
# Conver video to images 
# output files located at ./assets/image
# frameRate decide the fps """

import cv2
vidcap = cv2.VideoCapture('man-walking-with-a-laggage.mov')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("./assets/image"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 1 # frames per second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [ ]:
# Load a test image

# fpath = './assets/dog.jpg'
# input_img = Image.open(fpath)

# img = cv2.imread('./assets/dog.jpg', cv2.IMREAD_UNCHANGED)
# cv2_imshow(img)
 

In [ ]:
# Load video
vidcap = cv2.VideoCapture('president-on-tech-green.mov')

In [ ]:
# Functions

def getFrame(sec):
  vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
  hasFrames,img = vidcap.read()
  return (img, hasFrames)

def generateCv2img(cv2img, detection_processed, path = None, display = False):
  fontScale = 0.7 # font & text block ratio
  for box in detection_processed:
    id, label, conf = box[0], box[1], box[2]
    x1, y1, x2, y2 = box[3], box[4], box[5], box[6]
    cv2.rectangle(cv2img,(x1,y1),(x2,y2),(0,255,0),6)

    labelSize=cv2.getTextSize(label,cv2.FONT_HERSHEY_COMPLEX,fontScale,2)
    _x1 = x1
    _y1 = y1
    _x2 = _x1 + int(labelSize[0][0] * fontScale)
    _y2 = y1 - int(labelSize[0][1]* fontScale)
    cv2.rectangle(cv2img,(_x1,_y1),(_x2,_y2),(0,255,0),cv2.FILLED)
    cv2.putText(cv2img,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1)

  if path != None:
    cv2.imwrite(path + str(count) + ".jpg", cv2img)

  if display: 
    cv2_imshow(cv2img)

  return cv2img

  cv2.destroyAllWindows()


def generateNoPredCv2img(cv2img, path = None, display = False):
  # fontScale = 0.7 # font & text block ratio
  # for box in detection_processed:
  #   id, label, conf = box[0], box[1], box[2]
  #   x1, y1, x2, y2 = box[3], box[4], box[5], box[6]
  #   cv2.rectangle(cv2img,(x1,y1),(x2,y2),(0,255,0),6)

  #   labelSize=cv2.getTextSize(label,cv2.FONT_HERSHEY_COMPLEX,fontScale,2)
  #   _x1 = x1
  #   _y1 = y1
  #   _x2 = _x1 + int(labelSize[0][0] * fontScale)
  #   _y2 = y1 - int(labelSize[0][1]* fontScale)
  #   cv2.rectangle(cv2img,(_x1,_y1),(_x2,_y2),(0,255,0),cv2.FILLED)
  #   cv2.putText(cv2img,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1)

  if path != None:
    cv2.imwrite(path + str(count) + ".jpg", cv2img)

  if display: 
    cv2_imshow(cv2img)

  return cv2img

  cv2.destroyAllWindows()

# YOLOv3 (Darknet53) - template

In [ ]:
from detectors.yolov3 import YOLOv3_Darknet53

# Load the model
detector = YOLOv3_Darknet53(weights='model_weights/YOLOv3_VOC0712_Darknet53.h5')

# Parameters
sec = 0
fps = 24
frameRate = 1 / fps # interval between frames

count = 0
success = getFrame(sec)
images = []
cv2imgs = []
times = []
results = []
result_imgs = []

modelName = "yolov3dark"
path = './output/'+ modelName + str(fps) + '/'
noPredPath = './output/'+ modelName + str(fps) + 'NoPred/'


  # detection_raw = detector.detect(x_query, conf_threshold=detector.confidence_thresh_default)
# Convert the video to frames and make prediction
while success:
  count = count + 1
  sec = sec + frameRate
  sec = round(sec, 2)
  image ,success = getFrame(sec)
  
  if not success:
    continue

  # convert np array back to image
  cv2img = image
  cv2imgs.append(image)
  image = Image.fromarray(image) 
  images.append(image)

  # Time and make prediction
  x_query, x_meta = letterbox_image_padded(image, size=detector.model_img_size)
  start = time.time()
  detection_raw = detector.detect(x_query, conf_threshold=detector.confidence_thresh_default)
  end = time.time()
  detection_processed = decode_detection_raw(detection_raw, x_meta, detector.classes)
  
  # Generate and save the result image
  result_img = generateCv2img(cv2img, detection_processed, path = path, display = False)
  result_imgs.append(result_img)

  Generate no prediction images
  result_img = generateNoPredCv2img(cv2img, noPredPath, display=False)

  # Collect the result
  results.append(detection_processed)
  times.append(end - start)



# # Time summary    
summary = [max(times), min(times), sum(times)/len(times) ]
print("Prediction time list", times)
print("Max:", max(times))
print("Min:", min(times))
print("Avg:", sum(times)/len(times))

# # Save intervals 
data = json.dumps(times)

with open(path + 'intervals.json', 'w') as f:
    json.dump(data, f)
# Save summary
data = json.dumps(summary)
with open(path + 'time_summary.json', 'w') as f:
    json.dump(data, f)

  
# # Confidence summary
conf = []
for elm in results:
  if len(elm) != 0:
    conf.append(elm[0][2])

confFps = {}
confFps['rawConfList'] = conf
confFps['maxConf'] = max(conf)
confFps['minConf'] = min(conf)
confFps['avgConf'] = sum(conf)/len(conf)
print('max(conf)', confFps['maxConf'])
print('min(conf)', confFps['minConf'])
print('avgConf', confFps['avgConf'])


In [ ]:
results[110]

[(14, 'person', 0.9977402687072754, 67, 189, 137, 358),
 (14, 'person', 0.9973552227020264, 468, 71, 721, 457)]

# YOLOv3 (MobileNetV1)

In [ ]:
# from detectors.yolov3 import YOLOv3_MobileNetV1

# start = time.time()
# detector = YOLOv3_MobileNetV1(weights='model_weights/YOLOv3_VOC0712_MobileNetV1.h5')
# afterLoading = time.time()

# x_query, x_meta = letterbox_image_padded(input_img, size=detector.model_img_size)
# afterLetterbox = time.time()

# detection_raw = detector.detect(x_query, conf_threshold=detector.confidence_thresh_default)
# afterDetect = time.time()

# detection_processed = decode_detection_raw(detection_raw, x_meta, detector.classes)
# afterProcessResult = time.time()

# visualize_detection(input_img, detection_processed)
# afterVisualization = time.time()

# print("Load detector loading", afterLoading - start)
# print(" padding", afterLetterbox - afterLetterbox)
# print(" detection", afterDetect - afterLetterbox)
# print(" process result", afterProcessResult - afterDetect)
# print(" visualization", afterVisualization - afterProcessResult)





# SSD300 (VGG16)

In [ ]:
# from detectors.ssd import SSD300
# detector = SSD300(weights='model_weights/SSD_VOC0712_VGG16_300x300.h5')

# x_query, x_meta = letterbox_image_padded(input_img, size=detector.model_img_size)
# detection_raw = detector.detect(x_query, conf_threshold=detector.confidence_thresh_default)
# detection_processed = decode_detection_raw(detection_raw, x_meta, detector.classes)

# visualize_detection(input_img, detection_processed)



# SSD512 (VGG16)

In [ ]:
# from detectors.ssd import SSD512
# detector = SSD512(weights='model_weights/SSD_VOC0712_VGG16_512x512.h5')

# x_query, x_meta = letterbox_image_padded(input_img, size=detector.model_img_size)
# detection_raw = detector.detect(x_query, conf_threshold=detector.confidence_thresh_default)
# detection_processed = decode_detection_raw(detection_raw, x_meta, detector.classes)

# visualize_detection(input_img, detection_processed)

# Faster R-CNN (VGG16)

In [ ]:
from detectors.frcnn import FRCNN

# Load models and videos
detector = FRCNN().cuda(device=0).load('model_weights/FRCNN_VOC0712_VGG16.pth')
vidcap = cv2.VideoCapture('man-walking-with-a-laggage.mov')

def getFrame(sec):
  vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
  hasFrames,img = vidcap.read()
  return (img, hasFrames)

# Parameters
sec = 0
fps = 24
frameRate = 1 / fps # interval between frames
count = 0
success = getFrame(sec)
images = []
times = []
results = []

# int frames = 

# Convert the video to frames and make prediction
while success:
  count = count + 1
  sec = sec + frameRate
  sec = round(sec, 2)
  image ,success = getFrame(sec)

  if not success:
    continue

  # convert np array back to image
  image = Image.fromarray(image) 

  images.append(image)

  # Time and make prediction
  start = time.time()
  x_query, x_meta = letterbox_image_padded(image, size=detector.model_img_size)
  detection_raw = detector.detect(x_query, conf_threshold=detector.confidence_thresh_default)
  detection_processed = decode_detection_raw(detection_raw, x_meta, detector.classes)
  
  # Generate and save the result image
  plt.clf()
  plt.figure(figsize=(3, 3))
  plt.imshow(image)
  current_axis = plt.gca()
  for box in detection_processed:
      class_id = box[0]
      class_name = box[1]
      confidence = box[2]
      xmin, ymin, xmax, ymax = box[3:]
      color = colors[class_id]
      label = '{}: {:.2f}'.format(class_name, confidence)
      current_axis.add_patch(
          plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, color=color, fill=False, linewidth=2))
      current_axis.text(xmin, ymin, label, size='small', color='black', bbox={'facecolor': color, 'alpha': 1.0})
  plt.axis('off')
  plt.savefig('./output/frcnn' + str(fps) + '/' + str(count) + ".jpg")
  plt.close()
  
  results.append(detection_processed)
  # Display image and print the time
  # plt.show() 
  # print('Time for', count, ':', time.time() - start)
  times.append(time.time() - start)

# Print results
print("Number of frames:", len(times))
print("Prediction time list", times)
print("Max:", max(times))
print("Min:", min(times))
print("Avg:", sum(times)/len(times))

# print("Prediction time list", times[1:])
# print("Max:", max(times[1:]))
# print("Min:", min(times[1:]))
# print("Avg:", sum(times[1:])/len(times[1:]))

# Time data to json
timeFps24 = {}
timeFps24['rawTimeList'] = times
timeFps24['maxTime'] = max(times)
timeFps24['minTime'] = min(times)
timeFps24['avgTime'] = sum(times)/len(times)

data = json.dumps(timeFps24)

with open('frcnn-time-fps' + str(fps) + '.json', 'w') as f:
    json.dump(data, f)

# # Confidence json example
# confFps24 = {}
# confFps24['rawConfList'] = conf
# confFps24['maxConf'] = max(conf)
# confFps24['minConf'] = min(conf)
# confFps24['avgConf'] = sum(conf)/len(conf)

conf = []
for elm in results:
  if len(elm) != 0:
    conf.append(elm[0][2])
    
# Confidence 
confFps = {}
confFps['rawConfList'] = conf
confFps['maxConf'] = max(conf)
confFps['minConf'] = min(conf)
confFps['avgConf'] = sum(conf)/len(conf)
print('max(conf)', max(conf))
print('min(conf)', min(conf))

data = json.dumps(confFps)

with open('./results/frcnn-conf-fps' + str(fps) + '.json', 'w') as f:
    json.dump(data, f)

# data = json.dumps(confFps24)

# with open('frcnn-conf-fps24.json', 'w') as f:
#     json.dump(data, f)



Number of frames: 145
Prediction time list [0.7449469566345215, 0.7249050140380859, 0.7237453460693359, 0.7222223281860352, 0.7468395233154297, 0.7458624839782715, 0.7431554794311523, 0.7161104679107666, 0.7244644165039062, 0.7531154155731201, 0.7579562664031982, 0.7494015693664551, 0.7190966606140137, 0.7220158576965332, 0.7349593639373779, 0.7390279769897461, 0.736419677734375, 0.7216107845306396, 0.7232699394226074, 0.718327522277832, 0.7308573722839355, 0.7524247169494629, 0.7399032115936279, 0.7774264812469482, 0.7335524559020996, 0.7379856109619141, 0.7387127876281738, 0.7364380359649658, 0.7144253253936768, 0.7308437824249268, 0.7668163776397705, 0.8882791996002197, 0.75307297706604, 0.731184720993042, 0.7388408184051514, 0.7535130977630615, 0.8558692932128906, 0.7351124286651611, 0.7292594909667969, 0.7113137245178223, 0.7226529121398926, 0.7378325462341309, 0.713914155960083, 0.7206425666809082, 0.7228696346282959, 0.7376513481140137, 0.7201616764068604, 0.7406826019287109, 0.

<Figure size 432x288 with 0 Axes>

In [ ]:
print("Number of frames:", len(times))
print("Prediction time list", times)
print("Max:", max(times))
print("Min:", min(times))
print("Avg:", sum(times)/len(times))

Number of frames: 145
Prediction time list [0.7449469566345215, 0.7249050140380859, 0.7237453460693359, 0.7222223281860352, 0.7468395233154297, 0.7458624839782715, 0.7431554794311523, 0.7161104679107666, 0.7244644165039062, 0.7531154155731201, 0.7579562664031982, 0.7494015693664551, 0.7190966606140137, 0.7220158576965332, 0.7349593639373779, 0.7390279769897461, 0.736419677734375, 0.7216107845306396, 0.7232699394226074, 0.718327522277832, 0.7308573722839355, 0.7524247169494629, 0.7399032115936279, 0.7774264812469482, 0.7335524559020996, 0.7379856109619141, 0.7387127876281738, 0.7364380359649658, 0.7144253253936768, 0.7308437824249268, 0.7668163776397705, 0.8882791996002197, 0.75307297706604, 0.731184720993042, 0.7388408184051514, 0.7535130977630615, 0.8558692932128906, 0.7351124286651611, 0.7292594909667969, 0.7113137245178223, 0.7226529121398926, 0.7378325462341309, 0.713914155960083, 0.7206425666809082, 0.7228696346282959, 0.7376513481140137, 0.7201616764068604, 0.7406826019287109, 0.

In [ ]:
# Convert images back to video
import cv2
import numpy as np
import glob
 
img_array = []
for filename in  glob.glob('./output/frcnn' + str(24) + '/*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('test.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 24, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()


SyntaxError: ignored

In [ ]:
timeFps24 = {}
timeFps24['rawTimeList'] = times
timeFps24['maxTime'] = max(times)
timeFps24['minTime'] = min(times)
timeFps24['avgTime'] = sum(times)/len(times)

data = json.dumps(timeFps24)

with open('frcnn-time-fps24.json', 'w') as f:
    json.dump(data, f)


In [ ]:
# # >>> v.get(cv2.CAP_PROP_POS_MSEC)
# # 213400.0
# # Checking shows that this sets the point after the last frame (not before it), so the timestamp is indeed the exact total length of the stream:

# # >>> v.get(cv2.CAP_PROP_POS_FRAMES)
# # 5335.0
# # >>>> v.get(cv2.CAP_PROP_FRAME_COUNT)
# # 5335.0

# cap = cv2.VideoCapture('man-walking-with-a-laggage.mov')
# # vidcap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
# # cap.get(cv2.CAP_PROP_FPS)
# # vidcap.get(cv2.CAP_PROP_POS_FRAMEcv2.CAP_PROP_FPSS)

# old_fps = cap.get(cv2.CAP_PROP_FPS)      
# frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# duration = frame_count/old_fps
# new_fps = 24
# new_frame_count = int(duration * new_fps)

# print('fps = ' + str(fps))
# print('number of frames = ' + str(frame_count))
# print('duration (S) = ' + str(duration))
# print('new count = ' + str(new_frame_count))

In [ ]:
# # Convert images to video

# # img_array = []
# # for filename in glob.glob('C:/New folder/Images/*.jpg'):
# #     img = cv2.imread(filename)
# #     height, width, layers = img.shape
# #     size = (width,height)
# #     img_array.append(img)


# out = cv2.VideoWriter('test.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 24, size)
# for i in range(len(images)):
#     out.write(img_array[i])
# out.release()

In [ ]:
# Convert imgs to video from cv2imgs

size = cv2imgs[0].shape[:2]
out = cv2.VideoWriter('test.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 24, size)
for i in range(len(cv2imgs)):
    out.write(cv2imgs[i])
out.release()